# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [19]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

print(adult_dt)


       age          workclass  fnlwgt    education  education-num  \
0       39          State-gov   77516    Bachelors             13   
1       50   Self-emp-not-inc   83311    Bachelors             13   
2       38            Private  215646      HS-grad              9   
3       53            Private  234721         11th              7   
4       28            Private  338409    Bachelors             13   
...    ...                ...     ...          ...            ...   
32556   27            Private  257302   Assoc-acdm             12   
32557   40            Private  154374      HS-grad              9   
32558   58            Private  151910      HS-grad              9   
32559   22            Private  201490      HS-grad              9   
32560   52       Self-emp-inc  287927      HS-grad              9   

            marital-status          occupation    relationship    race  \
0            Never-married        Adm-clerical   Not-in-family   White   
1       Married-civ-spo

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [20]:
from sklearn.model_selection import train_test_split

# Create dataframe `X` with all columns except `income`
X = adult_dt.drop(columns=['income'])

# Create dataframe `Y` with the target data `income`
Y = adult_dt['income']

# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")


X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

*(Comment here.)*

The random_state parameter in the train_test_split function is used to control the shuffling applied to the data before the split is applied.

It is useful becauses using the same random_state ensures that the model can be tested and validated under the same conditions, making the results consistent and comparable.

If there bug, being able to reproduce the same split helps to debug more effectively.

When sharing with others, reproducible results ensure that they can replicate the findings and verify the methods.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler

# Specify which columns are numerical and which are categorical
numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Define the preprocessing pipelines for numerical and categorical features
numerical_transformer = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist', drop='if_binary'))
])

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# preprocessed = preprocessor.fit_transform(X_train, Y_train)

# print(f"Preprocessed Features Shape: {preprocessed.shape}")


## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [22]:
from sklearn.ensemble import RandomForestClassifier

# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),  # ColumnTransformer from the previous section
    ('classifier', RandomForestClassifier())  # RandomForestClassifier
])

# Fit the pipeline to the data
model_pipeline.fit(X_train, Y_train)

# Print pipeline steps to verify
print(model_pipeline)


Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('scaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [23]:
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score
from sklearn.model_selection import train_test_split, cross_validate

# Define the scoring metrics
scoring = {
    'neg_log_loss': make_scorer(log_loss, greater_is_better=False, needs_proba=True),
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation
cv_results = cross_validate(model_pipeline, X_train, Y_train, cv=5, scoring=scoring, return_train_score=True)

# Print the results
print("Cross-Validation Results:")
for key in scoring.keys():
    train_score_mean = cv_results[f'train_{key}'].mean()
    valid_score_mean = cv_results[f'test_{key}'].mean()
    print(f"{key}: Train - {train_score_mean:.4f}, Validation - {valid_score_mean:.4f}")


C:\Users\v-sharmashwe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_scorer.py:610: FutureWarning: The `needs_threshold` and `needs_proba` parameter are deprecated in version 1.4 and will be removed in 1.6. You can either let `response_method` be `None` or set it to `predict` to preserve the same behaviour.
  warnings.warn(
C:\Users\v-sharmashwe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
C:\Users\v-sharmashwe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transf

Cross-Validation Results:
neg_log_loss: Train - -0.0815, Validation - -0.3820
roc_auc: Train - 1.0000, Validation - 0.9038
accuracy: Train - 1.0000, Validation - 0.8529
balanced_accuracy: Train - 1.0000, Validation - 0.7748


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [24]:
# Convert results to DataFrame
cv_df = pd.DataFrame(cv_results)

# Sorting by negative log loss of the test set
sorted_cv_df = cv_df.sort_values(by='test_neg_log_loss')

print("Fold-level Cross-Validation Results:")
print(sorted_cv_df)


Fold-level Cross-Validation Results:
   fit_time  score_time  test_neg_log_loss  train_neg_log_loss  test_roc_auc  \
4  8.074976    0.173189          -0.407574           -0.081217      0.902307   
1  8.657535    0.208905          -0.386670           -0.081101      0.902921   
2  9.381928    0.202697          -0.380224           -0.081295      0.902980   
0  8.274936    0.165288          -0.370144           -0.081379      0.904954   
3  8.254228    0.160418          -0.365609           -0.082294      0.905995   

   train_roc_auc  test_accuracy  train_accuracy  test_balanced_accuracy  \
4            1.0       0.854322        0.999945                0.775239   
1            1.0       0.848432        1.000000                0.769042   
2            1.0       0.852567        1.000000                0.773157   
0            1.0       0.850844        1.000000                0.775551   
3            1.0       0.858491        0.999945                0.781067   

   train_balanced_accuracy  
4 

Calculate the mean of each metric. 

In [25]:
# Calculate the mean of each metric 
mean_scores = sorted_cv_df.mean() 
print("Mean of each metric:") 
print(mean_scores)

Mean of each metric:
fit_time                   8.528721
score_time                 0.182099
test_neg_log_loss         -0.382044
train_neg_log_loss        -0.081457
test_roc_auc               0.903831
train_roc_auc              1.000000
test_accuracy              0.852931
train_accuracy             0.999978
test_balanced_accuracy     0.774811
train_balanced_accuracy    0.999955
dtype: float64


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [26]:
# Make predictions on the test data
y_pred_proba = model_pipeline.predict_proba(X_test)
y_pred = model_pipeline.predict(X_test)

# Calculate performance metrics
metrics = {
    'neg_log_loss': -log_loss(Y_test, y_pred_proba),
    'roc_auc': roc_auc_score(Y_test, y_pred_proba[:, 1]),
    'accuracy': accuracy_score(Y_test, y_pred),
    'balanced_accuracy': balanced_accuracy_score(Y_test, y_pred)
}

# Display results as a dictionary
print("Performance Metrics on Testing Data:")
print(metrics)


Performance Metrics on Testing Data:
{'neg_log_loss': -0.37874685976082434, 'roc_auc': np.float64(0.8996022809284402), 'accuracy': 0.8566895280990889, 'balanced_accuracy': np.float64(0.776914236789217)}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

(Answer here.)

Scikit-learn expects the target labels for binary classification to be binary, i.e., 0 or 1. Converting '>50K' to 1 immediately ensures that the target variable is in the format expected by the evaluation metrics and classification algorithms.

In essence, it prepares the data in a way that is most compatible with machine learning models and evaluation techniques, making the process efficient and less error-prone.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.